In [1]:
import pandas as pd

DATA_FILE_CSV_PATH = "./flight_data.csv"

df_raw = pd.read_csv(f"{DATA_FILE_CSV_PATH}", sep=',', header=None, low_memory=False)

Take a glance at the data.

In [2]:
df_raw.head()

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,UNIQUE_CARRIER,FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,...,AIR_TIME,FLIGHTS,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,Unnamed: 42
1,1,1,3,7,2016-01-03,F9,694,11292,1129202,30325,...,87.0,1.0,692.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1,3,7,2016-01-03,F9,809,14027,1402702,34027,...,224.0,1.0,1679.0,19.0,0.0,0.0,0.0,0.0,NaN,NaN
3,1,1,3,7,2016-01-03,F9,907,15356,1535602,35356,...,60.0,1.0,373.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1,3,7,2016-01-03,F9,908,14492,1449202,34492,...,57.0,1.0,373.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Do you see *any* oddities? Column names are in fact in the first row! Let's name the columns and drop the row.

In [3]:
df_raw.columns = df_raw.iloc[0]
df_raw.drop(df_raw.index[0], inplace=True)

See if any sring type columns exist, which can be nuisance for Random Forest

In [4]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1856061 entries, 1 to 1856061
Data columns (total 43 columns):
QUARTER                  object
MONTH                    object
DAY_OF_MONTH             object
DAY_OF_WEEK              object
FL_DATE                  object
UNIQUE_CARRIER           object
FL_NUM                   object
ORIGIN_AIRPORT_ID        object
ORIGIN_AIRPORT_SEQ_ID    object
ORIGIN_CITY_MARKET_ID    object
ORIGIN                   object
ORIGIN_CITY_NAME         object
ORIGIN_STATE_ABR         object
ORIGIN_STATE_NM          object
DEST_AIRPORT_ID          object
DEST_AIRPORT_SEQ_ID      object
DEST_CITY_MARKET_ID      object
DEST                     object
DEST_CITY_NAME           object
DEST_STATE_ABR           object
DEST_STATE_NM            object
CRS_DEP_TIME             object
DEP_TIME                 object
DEP_DELAY                object
DEP_DELAY_NEW            object
WHEELS_ON                object
TAXI_IN                  object
CRS_ARR_TIME           

Next, check on how pervasive Not A Number (NaN), converted from null value in the original data source upon importation into Pandas, are.

In [5]:
# Display the portion of NaNs for every column in percentage
(df_raw.isnull().sum()/df_raw.shape[0]) * 100

0
QUARTER                    0.000000
MONTH                      0.000000
DAY_OF_MONTH               0.000000
DAY_OF_WEEK                0.000000
FL_DATE                    0.000000
UNIQUE_CARRIER             0.000000
FL_NUM                     0.000000
ORIGIN_AIRPORT_ID          0.000000
ORIGIN_AIRPORT_SEQ_ID      0.000000
ORIGIN_CITY_MARKET_ID      0.000000
ORIGIN                     0.000000
ORIGIN_CITY_NAME           0.000000
ORIGIN_STATE_ABR           0.000000
ORIGIN_STATE_NM            0.000000
DEST_AIRPORT_ID            0.000000
DEST_AIRPORT_SEQ_ID        0.000000
DEST_CITY_MARKET_ID        0.000000
DEST                       0.000000
DEST_CITY_NAME             0.000000
DEST_STATE_ABR             0.000000
DEST_STATE_NM              0.000000
CRS_DEP_TIME               0.000000
DEP_TIME                   1.429048
DEP_DELAY                  1.429048
DEP_DELAY_NEW              1.429048
WHEELS_ON                  1.512019
TAXI_IN                    1.512019
CRS_ARR_TIME              

Hang on a second! Unusually high number of NaNs in the last seven (7) columns! Take a glance at them now.

In [6]:
df_raw.iloc[:,-7:]

,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,FIRST_DEP_TIME,Unnamed: 42
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19.0,0.0,0.0,0.0,0.0,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2.0,0.0,18.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...
1856057,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1856058,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1856059,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1856060,NaN,NaN,NaN,NaN,NaN,NaN,NaN


They don't look THAT relevant, don't they? We will drop them all.

In [7]:
df_raw.drop(df_raw.columns[-7:], axis=1, inplace=True)

Let's take a look at a separate DataFrame on and after the 'DEP_TIME' column to check for NaNs

In [54]:
split_column_idx = df_raw.columns.get_loc('DEP_TIME')

df_keep = df_raw.iloc[:,:split_column_idx]
df_examine = df_raw.iloc[:,split_column_idx:]

In [56]:
df_examine.head()

,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE
1,1524.0,-1.0,0.0,1807.0,8.0,1820,1815.0,-5.0,0.0,115.0,111.0,87.0,1.0,692.0
2,744.0,44.0,44.0,940.0,8.0,929,948.0,19.0,19.0,269.0,244.0,224.0,1.0,1679.0
3,1858.0,13.0,13.0,2006.0,7.0,2015,2013.0,-2.0,0.0,90.0,75.0,60.0,1.0,373.0
4,2054.0,-1.0,0.0,2208.0,7.0,2220,2215.0,-5.0,0.0,85.0,81.0,57.0,1.0,373.0
5,1252.0,2.0,2.0,1349.0,51.0,1420,1440.0,20.0,20.0,150.0,168.0,107.0,1.0,693.0


Need to conver 'object' to 'float' to run general arithmetic analysis.

In [62]:
pd.options.mode.chained_assignment = None  # default='warn'
# Otherwise the following warning appear
# /home/to/miniconda3/envs/us_flight_delay/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
# A value is trying to be set on a copy of a slice from a DataFrame.
# Try using .loc[row_indexer,col_indexer] = value instead
#
# See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
#   This is separate from the ipykernel package so we can avoid doing imports until

for col in df_examine:
    #df_examine[col] = pd.to_numeric(df_examine[col], errors='coerce')
    df_examine[col] = df_examine[col].astype(float)
    
df_examine.describe()

/home/to/miniconda3/envs/us_flight_delay/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE
count,1.829537e+06,1.829537e+06,1.829537e+06,1.827997e+06,1.827997e+06,1.856061e+06,1.827997e+06,1.824403e+06,1.824403e+06,1.856058e+06,1.824403e+06,1.824403e+06,1856061.0,1.856061e+06
mean,1.334419e+03,9.812064e+00,1.267238e+01,1.471640e+03,7.562686e+00,1.494374e+03,1.476238e+03,4.206204e+00,1.259892e+01,1.463799e+02,1.408941e+02,1.169484e+02,1.0,8.523126e+02
std,5.004642e+02,4.166410e+01,4.064121e+01,5.270121e+02,6.047238e+00,5.133614e+02,5.314493e+02,4.389959e+01,4.031154e+01,7.673182e+01,7.587500e+01,7.374921e+01,0.0,6.198460e+02
min,1.000000e+00,-6.000000e+01,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-1.070000e+02,0.000000e+00,2.000000e+01,1.400000e+01,7.000000e+00,1.0,2.500000e+01
25%,9.200000e+02,-5.000000e+00,0.000000e+00,1.053000e+03,4.000000e+00,1.109000e+03,1.057000e+03,-1.400000e+01,0.000000e+00,9.000000e+01,8.400000e+01,6.200000e+01,1.0,3.910000e+02
50%,1.329000e+03,-2.000000e+00,0.000000e+00,1.510000e+03,6.000000e+00,1.522000e+03,1.514000e+03,-6.000000e+00,0.000000e+00,1.270000e+02,1.220000e+02,9.800000e+01,1.0,6.790000e+02
75%,1.741000e+03,7.000000e+00,7.000000e+00,1.914000e+03,9.000000e+00,1.920000e+03,1.919000e+03,7.000000e+00,7.000000e+00,1.790000e+02,1.740000e+02,1.480000e+02,1.0,1.096000e+03
max,2.400000e+03,2.040000e+03,2.040000e+03,2.400000e+03,2.500000e+02,2.359000e+03,2.400000e+03,2.028000e+03,2.028000e+03,7.050000e+02,7.780000e+02,7.230000e+02,1.0,4.983000e+03


Mmmm, numbers in scientific notation are much harder to grasp... Let's set float to 2 decimal points

In [63]:
pd.options.display.float_format = '{:.2f}'.format

In [64]:
df_examine.describe()

,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE
count,1829537.00,1829537.00,1829537.00,1827997.00,1827997.00,1856061.00,1827997.00,1824403.00,1824403.00,1856058.00,1824403.00,1824403.00,1856061.00,1856061.00
mean,1334.42,9.81,12.67,1471.64,7.56,1494.37,1476.24,4.21,12.60,146.38,140.89,116.95,1.00,852.31
std,500.46,41.66,40.64,527.01,6.05,513.36,531.45,43.90,40.31,76.73,75.87,73.75,0.00,619.85
min,1.00,-60.00,0.00,1.00,1.00,1.00,1.00,-107.00,0.00,20.00,14.00,7.00,1.00,25.00
25%,920.00,-5.00,0.00,1053.00,4.00,1109.00,1057.00,-14.00,0.00,90.00,84.00,62.00,1.00,391.00
50%,1329.00,-2.00,0.00,1510.00,6.00,1522.00,1514.00,-6.00,0.00,127.00,122.00,98.00,1.00,679.00
75%,1741.00,7.00,7.00,1914.00,9.00,1920.00,1919.00,7.00,7.00,179.00,174.00,148.00,1.00,1096.00
max,2400.00,2040.00,2040.00,2400.00,250.00,2359.00,2400.00,2028.00,2028.00,705.00,778.00,723.00,1.00,4983.00


What's obvious is that the FLIGHTS column is populated 1 in every row, so we eliminate that.

In [71]:
df_examine.drop(df_examine['FLIGHTS'], axis=1, inplace=True)

No data set is immaculately ready for number crunching. Let's compare two techniques to remove outliers.

1. Turkey Fences, and
2. Z-Score

In [89]:
import numpy as np

def outliers_iqr(data):
    q1, q3 = np.percentile(data, [25, 75])
    iqr = q3 - q1
    lower_bound = q1 - (iqr * 1.5)
    upper_bound = q3 + (iqr * 1.5)
    return np.where((data > upper_bound) | (data < lower_bound))

#t = outliers_iqr(df_examine['DEP_DELAY'])

for col in df_examine:
    print(col)
    print(outliers_iqr(df_examine[col]))

(array([], dtype=int64),)
DEP_TIME
(array([], dtype=int64),)
DEP_DELAY
(array([], dtype=int64),)
DEP_DELAY_NEW
(array([], dtype=int64),)
WHEELS_ON
(array([], dtype=int64),)
TAXI_IN
(array([], dtype=int64),)
CRS_ARR_TIME
(array([], dtype=int64),)
ARR_TIME
(array([], dtype=int64),)
ARR_DELAY
(array([], dtype=int64),)
ARR_DELAY_NEW
(array([], dtype=int64),)
CRS_ELAPSED_TIME
(array([], dtype=int64),)
ACTUAL_ELAPSED_TIME
(array([], dtype=int64),)
AIR_TIME
(array([], dtype=int64),)
DISTANCE
(array([     18,      43,      50, ..., 1855946, 1855991, 1856016]),)


Mmm, the 'DISTANCE' column looks a bit fishy. the output tupple prints row numbers in suspect. Get rid of them!

In [9]:
length(outliers_iqr(df_examine[col])))

,22,23,24,25,26,27,28,29,30,31,32,33
count,1829538,1829538,1829538,1827998,1827998,1856062,1827998,1824404,1824404,1856059,1824404,1824404
unique,1441,1144,1092,1441,187,1432,1441,1200,1098,534,704,680
top,555.0,-3.0,0.0,1630.0,5.0,1700,1632.0,-9.0,0.0,80.0,80.0,64.0
freq,4177,144061,1166483,2165,299672,6292,2149,54904,1181756,39735,14768,15922


In [32]:
df_examine = df_raw.iloc[:,22:34]
df_examine

,22,23,24,25,26,27,28,29,30,31,32,33
0,DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,ARR_DELAY_NEW,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME
1,1524.0,-1.0,0.0,1807.0,8.0,1820,1815.0,-5.0,0.0,115.0,111.0,87.0
2,744.0,44.0,44.0,940.0,8.0,929,948.0,19.0,19.0,269.0,244.0,224.0
3,1858.0,13.0,13.0,2006.0,7.0,2015,2013.0,-2.0,0.0,90.0,75.0,60.0
4,2054.0,-1.0,0.0,2208.0,7.0,2220,2215.0,-5.0,0.0,85.0,81.0,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1856057,525.0,0.0,0.0,705.0,6.0,716,711.0,-5.0,0.0,111.0,106.0,88.0
1856058,1341.0,-1.0,0.0,1442.0,7.0,1446,1449.0,3.0,3.0,124.0,128.0,105.0
1856059,1531.0,3.0,3.0,1812.0,4.0,1845,1816.0,-29.0,0.0,137.0,105.0,89.0
1856060,1117.0,2.0,2.0,1254.0,5.0,1302,1259.0,-3.0,0.0,107.0,102.0,71.0


In [19]:
print (pd.to_numeric(df_examine['DEP_TIME'], errors='coerce'))

KeyError: 'DEP_TIME'

In [12]:
df_examine.values.astype(float)

ValueError: could not convert string to float: 'DEP_TIME'